<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Импортируем библиотеки, которые нам понадобятся для дальнейшей работы:

In [1]:
import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Откроем наш датасет. Выведем на экран первые пять объектов, общую информацию о датасете и список характерных значений.

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

display(df.head())
print('--------------------------------------------------')
df.info()
print('--------------------------------------------------')
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
--------------------------------------------------


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Согласно документации к данным:
- **Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.
- **Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

В результате первоначального ознакомления с датасетом, мы можем отметить, что датасет состоит из 5 тыс. объектов и 5 признаков. Столбец `Страховые выплаты` будет являться целевым признаком. Пропусков не обнаружено. Далее скорректируем тип данных и проверим наш датасет на дубликаты.

Для начала, скорерктируем тип данных для столбцов `Возраст` и `Зарплата`.

In [3]:
df['Возраст'] = df['Возраст'].astype('int')
df['Зарплата'] = df['Зарплата'].astype('int')

Проверим наши данные на дубликаты:

In [4]:
df.duplicated().sum()

153

Было обнаружено 153 дубликата. Почистим наш датасет.

In [5]:
df = df.drop_duplicates().reset_index(drop=True)

Следующим шагом разделим наш датасет на признаки и целевой признак.

In [6]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

Открыли, изучили и провели предобработку нашего датасета. Можно двигаться дальше!

## Умножение матриц

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново)

a. Изменится. 

b. Не изменится.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** b. После умножения признаков на обратимую матрицу качество линейной регрессии не изменится. Но поменяются веса для новых признаков. 

**Обоснование:**

Пусть

$$
X_1 = X P
$$
$$
w_1 = ((X_1^T X_1)^{-1} X_1^T y 
$$

, тогда

$$
w_1 = ((X P)^T X P)^{-1} (X P)^T y
$$

Применим свойство матриц $(A B)^T = B^T A^T$:

$$
w_1 = (P^T X^T X P)^{-1} P^T X^T y
$$

Матрицы $X$ и $X^T$ прямоугольные, поэтому для них не существует обратных матриц, а матрица $P$ - квдратная. Вынести из формулы множители в виде обратных матриц $P$ можно:

$$
w_1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Т.к.  $(P^T)^{-1} P^T$ - это единичная матрица, то мы можем её сократить, тогда остаётся следующее:

$$
w_1 = P^{-1} (X^T X)^{-1} X^T y
$$

Т.к. $w = (X^T X)^{-1} X^T y$ - это веса, из этого следует:

$$
w_1 = P^{-1} w
$$

Подставим получившееся значение в формулу предсказаний:

$$
a_1 = X_1 w_1
$$
$$
a_1 = X P P^{-1}w
$$
$$
a_1 = X w
$$

## Алгоритм преобразования

Создадим рандомную матрицу размером 4х4 и сохраним её в переменной `p_matrix`.

In [7]:
p_matrix = np.random.randint(100, size=(4, 4))
p_matrix

array([[95, 13, 63, 65],
       [23, 91, 21, 28],
       [80,  3,  3, 95],
       [97, 37, 36, 58]])

Проверим нашу матрицу на обратимость:

In [8]:
np.linalg.inv(p_matrix)

array([[-0.01487626, -0.01109831, -0.00674   ,  0.03306913],
       [-0.00620044,  0.00978109, -0.00187719,  0.00530156],
       [ 0.0273489 ,  0.00557491, -0.00643648, -0.02279845],
       [ 0.01185953,  0.00886102,  0.01646464, -0.02729516]])

Отлично! Наша матрица обратима.

Далее чтобы практически доказать, что метрика качества не изменится, вручную посчитаем $a$ (до умножения признаков на обратимую матрицу) и $a_1$ (после умножения).

**До умножения:**

In [9]:
w = ((np.linalg.inv(features.T.dot(features))).dot(features.T)).dot(target)
a = features.dot(w)

**После умножения:**

In [10]:
features_1 = features.dot(p_matrix)
w_1 = ((np.linalg.inv(features_1.T.dot(features_1))).dot(features_1.T)).dot(target)
a_1 = features_1.dot(w_1)

**Итог:**

In [11]:
a

0       0.288795
1       0.588191
2       0.435566
3      -0.092785
4       0.307588
          ...   
4842    0.143509
4843    0.134366
4844   -0.023962
4845   -0.053629
4846    0.088639
Length: 4847, dtype: float64

In [12]:
a_1

0       0.288796
1       0.588191
2       0.435566
3      -0.092785
4       0.307588
          ...   
4842    0.143509
4843    0.134367
4844   -0.023962
4845   -0.053629
4846    0.088639
Length: 4847, dtype: float64

В итоге мы видим, что значение предсказаний после умножения признаков на обратимую матрицу не изменились! 

## Проверка алгоритма

**До преобразования:**

In [13]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
r2 = r2_score(target, predictions)
print(f'Метрика R2 до преобразования: {r2}')

Метрика R2 до преобразования: 0.4302010046633359


**После преобразования:**

In [14]:
features_p = features.dot(p_matrix)
model = LinearRegression()
model.fit(features_p, target)
predictions = model.predict(features_p)
r2_1 = r2_score(target, predictions)
print(f'Метрика R2 после преобразования: {r2_1}')

Метрика R2 после преобразования: 0.43020100466336275


## Вывод

В данном проекте перед нами стояла задача доказать, что при умножении признаков на обратимую матрицу качество линейной регрессии не изменится.

Во втором пункте нашего проекта мы доказала это теорией, в третьем провели это практически, подставив наши данные в формулу предсказания. А в заключительном пункте мы на примере реального датасета подтвердили, что качество линейной регрессии не изменилось.

Также здесь стоит отметить, что с помощью данного метода можно легко зашифровать личные данные пользователей, и это никак не повлияет на результаты предсказания. 